In [ ]:
# get networks

queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    subnetworks
    FROM google.compute.networks WHERE project = '{project}'
    """
    for project in projects
]

networks_df = (lambda df: df[df['name'] != 'null'])(run_stackql_queries(queries))
networks_df

In [ ]:
# network peerings
networks_subset_df = networks_df[networks_df['peerings'] != 'null'][['name', 'peerings']].copy()
network_peerings_df = explode_json_list_col(networks_subset_df, 'peerings', 'peering')
network_peerings_df

In [ ]:
# get project iam bindings

queries = [
    f"""
    SELECT role, condition, members, '{project}' as project
    FROM google.cloudresourcemanager.projects_iam_policies
    WHERE projectsId = '{project}'
    """
    for project in projects
]

_bindings_df = run_stackql_queries(queries)

project_bindings = (explode_json_list_col(_bindings_df, 'members', 'member')
 .assign(member_type=lambda x: x['member'].str.split(':', expand=True)[0],
         member_email=lambda x: x['member'].str.split(':', expand=True)[1])
)
project_bindings

In [ ]:
args_json = json.dumps({"project": projects[0]})

In [ ]:
%%stackql
SELECT * FROM google.compute.regions WHERE project = '{project}'

In [ ]:
# cloud functions
queries = [
    f"""
    SELECT locationId,
    '{project}' as project
    FROM google.cloudfunctions.locations WHERE projectsId = '{project}'
    """
    for project in projects
]

locations_df = run_stackql_queries(queries)
locations = locations_df['locationId'].tolist()

queries = [
    f"""
    SELECT *,
    '{project}' as project,
    '{location}' as location
    FROM google.cloudfunctions.functions WHERE projectsId = '{project}' AND locationsId = '{location}'
    """
    for project in projects
    for location in locations
]
functions_df = run_stackql_queries(queries)
functions_df